In [1]:
import torch as t
from typing import Union
from torch import nn
import torch.nn.functional as F
import plotly.express as px
import plotly.graph_objects as go
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange
from fancy_einsum import einsum
import os
from tqdm.auto import tqdm
from torchvision import transforms, datasets
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader, TensorDataset
import wandb
import w5d1_utils
import w5d1_tests

import sys 
sys.path.append('../common_modules')

import cnn_modules

In [8]:
def conv_transpose1d_minimal(x: t.Tensor, weights: t.Tensor) -> t.Tensor:
    '''Like torch's conv_transpose1d using bias=False and all other keyword arguments left at their default values.

    x: shape (batch, in_channels, width)
    weights: shape (in_channels, out_channels, kernel_width)

    Returns: shape (batch, out_channels, output_width)
    '''
    in_channels, out_channels, kernel_width = weights.shape
    
    x = cnn_modules.pad1d(x, left=kernel_width-1, right=kernel_width-1, pad_value=0)
    mod_kernel = rearrange(t.flip(weights, dims=[2]), 'in out width -> out in width')
    out = cnn_modules.conv1d(x, mod_kernel)

    return out

w5d1_tests.test_conv_transpose1d_minimal(conv_transpose1d_minimal)

All tests in `test_conv1d_minimal` passed!


In [14]:
def fractional_stride_1d(x, stride: int = 1):
    '''Returns a version of x suitable for transposed convolutions, i.e. "spaced out" with zeros between its values.
    This spacing only happens along the last dimension.

    x: shape (batch, in_channels, width)

    Example: 
        x = [[[1, 2, 3], [4, 5, 6]]]
        stride = 2
        output = [[[1, 0, 2, 0, 3], [4, 0, 5, 0, 6]]]
    '''

    if stride == 1:
        return x
    
    batch_size, in_channels, in_width = x.shape
    out_width = in_width * stride - 1
    out = t.zeros(batch_size, in_channels, out_width, dtype=t.int64)
    out[:,:,::2] = x
    
    return out

w5d1_tests.test_fractional_stride_1d(fractional_stride_1d)

# def conv_transpose1d(x, weights, stride: int = 1, padding: int = 0) -> t.Tensor:
#     '''Like torch's conv_transpose1d using bias=False and all other keyword arguments left at their default values.

#     x: shape (batch, in_channels, width)
#     weights: shape (in_channels, out_channels, kernel_width)

#     Returns: shape (batch, out_channels, output_width)
#     '''
#     return fractional_stride_1d

# w5d1_tests.test_conv_transpose1d(conv_transpose1d)

torch.int64
stride: 2
tensor([[[1, 2, 3],
         [4, 5, 6]]])
tensor([[[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]])
tensor([[[1, 0, 2, 0, 3],
         [4, 0, 5, 0, 6]]])
All tests in `test_fractional_stride_1d` passed!
